In [1]:
from datasets import load_dataset
import numpy as np
from transformers import pipeline,AutoTokenizer,TrainingArguments,AutoModelForSequenceClassification,Trainer
from pprint import pprint
from torchinfo import summary
from datasets import load_metric
import json


In [6]:
raw_datasets = load_dataset("glue","rte")
raw_datasets.data["train"]

MemoryMappedTable
sentence1: string
sentence2: string
label: int64
idx: int32
----
sentence1: [["No Weapons of Mass Destruction Found in Iraq Yet.","A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.","Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.","Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.","A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keywort

In [7]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_fn(batch):
    return tokenizer(batch['sentence1'],
                     batch['sentence2'],
                     truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_fn,batched=True)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [30]:
training_args = TrainingArguments(
    'trainer',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3,
)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)
summary(model)
metric = load_metric("glue","sst2")
def compute_metrics(logits_and_labels):
    logits,labels = logits_and_labels
    predictions = np.argmax(logits,axis=1)
    return metric.compute(predictions=predictions,references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
trainer.train()

  0%|          | 0/936 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 0.689373791217804, 'eval_accuracy': 0.5342960288808665, 'eval_runtime': 2.0539, 'eval_samples_per_second': 134.864, 'eval_steps_per_second': 17.041, 'epoch': 1.0}
{'loss': 0.6837, 'learning_rate': 2.3290598290598293e-05, 'epoch': 1.6}


  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 0.7390849590301514, 'eval_accuracy': 0.5703971119133574, 'eval_runtime': 2.0748, 'eval_samples_per_second': 133.506, 'eval_steps_per_second': 16.869, 'epoch': 2.0}


  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 0.9290688037872314, 'eval_accuracy': 0.6101083032490975, 'eval_runtime': 2.044, 'eval_samples_per_second': 135.518, 'eval_steps_per_second': 17.123, 'epoch': 3.0}
{'train_runtime': 218.9793, 'train_samples_per_second': 34.113, 'train_steps_per_second': 4.274, 'train_loss': 0.5782533140264006, 'epoch': 3.0}


TrainOutput(global_step=936, training_loss=0.5782533140264006, metrics={'train_runtime': 218.9793, 'train_samples_per_second': 34.113, 'train_steps_per_second': 4.274, 'train_loss': 0.5782533140264006, 'epoch': 3.0})

In [33]:
trainer.save_model("my_trained_model")
newmodel = pipeline('text-classification',model="my_trained_model",device=0)


[{'label': 'LABEL_0', 'score': 0.823796272277832}]

In [34]:
raw_datasets.data['train'][0]
newmodel('look at my horse.my horse is amazing')

newmodel("i love food. food is my love")

[{'label': 'LABEL_0', 'score': 0.9223436117172241}]